In [1]:
!pip install mediapipe opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [3]:
import cv2
import mediapipe as mp
import math

In [20]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
def calculate_angle(point1, point2, point3):
    x1, y1 = point1.x, point1.y
    x2, y2 = point2.x, point2.y
    x3, y3 = point3.x, point3.y

    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    return angle if angle >= 0 else angle + 360

REFERENCE_ANGLES = {
    "left_elbow": 160,
    "right_elbow": 160,
    "left_knee": 170,
    "right_knee": 170
}
ANGLE_THRESHOLD = 15

video_path = '/content/shoot_bow.avi'
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_path = '/content/processed_video11.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)


    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        landmarks = results.pose_landmarks.landmark
        left_elbow_angle = calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER],
                                           landmarks[mp_pose.PoseLandmark.LEFT_ELBOW],
                                           landmarks[mp_pose.PoseLandmark.LEFT_WRIST])
        right_elbow_angle = calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER],
                                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW],
                                            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST])
        left_knee_angle = calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP],
                                          landmarks[mp_pose.PoseLandmark.LEFT_KNEE],
                                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE])
        right_knee_angle = calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP],
                                           landmarks[mp_pose.PoseLandmark.RIGHT_KNEE],
                                           landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE])

        recommendations = []
        if abs(left_elbow_angle - REFERENCE_ANGLES["left_elbow"]) > ANGLE_THRESHOLD:
            recommendations.append("Adjust left elbow position")
        if abs(right_elbow_angle - REFERENCE_ANGLES["right_elbow"]) > ANGLE_THRESHOLD:
            recommendations.append("Adjust right elbow position")
        if abs(left_knee_angle - REFERENCE_ANGLES["left_knee"]) > ANGLE_THRESHOLD:
            recommendations.append("Adjust left knee position")
        if abs(right_knee_angle - REFERENCE_ANGLES["right_knee"]) > ANGLE_THRESHOLD:
            recommendations.append("Adjust right knee position")

        y_position = 50
        for recommendation in recommendations:
            cv2.putText(frame, recommendation, (10, y_position), cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 0, 255), 2, cv2.LINE_AA)
            y_position += 30

    out.write(frame)

cap.release()
out.release()
print("Processed video saved with recommendations at:", output_path)

Processed video saved with recommendations at: /content/processed_video11.mp4


Processed video saved to: /processed_video.mp4
